## Scenario 3: Multiple data scientists working on multiple ML models

MLflow setup:
* Tracking server: yes, remote server (EC2).
* Backend store: postgresql database.
* Artifacts store: s3 bucket.

The experiments can be explored by accessing the remote server.

The exampe uses AWS to host a remote server. In order to run the example you'll need an AWS account. Follow the steps described in the file `mlflow_on_aws.md` to create a new AWS account and launch the tracking server. 

In [31]:
import mlflow
import os

os.environ["AWS_PROFILE"] = "dev" # fill in with your AWS profile. More info: https://docs.aws.amazon.com/sdk-for-java/latest/developer-guide/setup.html#setup-credentials

TRACKING_SERVER_HOST = "3.87.201.140" # fill in with the public DNS of the EC2 instance
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

In [32]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://3.87.201.140:5000'


In [33]:
mlflow.search_experiments() # list_experiments API has been removed, you can use search_experiments instead.()

[<Experiment: artifact_location='s3://mlflow-artifacts-remote-ruoke/1', creation_time=1751446065678, experiment_id='1', last_update_time=1751446065678, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='s3://mlflow-artifacts-remote-ruoke/0', creation_time=1751446002748, experiment_id='0', last_update_time=1751446002748, lifecycle_stage='active', name='Default', tags={}>]

In [34]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, name="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/07/02 17:27:21 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: 's3://mlflow-artifacts-remote-ruoke/1/70a7a1df1530441dbe69e08a23ea58f8/artifacts'
🏃 View run upbeat-squid-159 at: http://3.87.201.140:5000/#/experiments/1/runs/70a7a1df1530441dbe69e08a23ea58f8
🧪 View experiment at: http://3.87.201.140:5000/#/experiments/1


In [29]:
mlflow.search_experiments()

[<Experiment: artifact_location='s3://mlflow-artifacts-remote-ruoke/1', creation_time=1751446065678, experiment_id='1', last_update_time=1751446065678, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='s3://mlflow-artifacts-remote-ruoke/0', creation_time=1751446002748, experiment_id='0', last_update_time=1751446002748, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [24]:
from mlflow.tracking import MlflowClient
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

client = MlflowClient(f"http://{TRACKING_SERVER_HOST}:5000")

In [30]:

run_id='a297a9bc617941519486bac78bd4567a'
model_uri = f"runs:/{run_id}/models"
print(f"Using model URI: {model_uri}")
#register model
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-xgboost")


Using model URI: runs:/a297a9bc617941519486bac78bd4567a/models


Registered model 'nyc-taxi-xgboost' already exists. Creating a new version of this model...
2025/07/02 17:07:59 WARNING mlflow.tracking._model_registry.fluent: Run with id a297a9bc617941519486bac78bd4567a has no artifacts at artifact path 'models', registering model based on models:/m-cffe91d66ddc4191b7e993c1dd0e2daf instead
2025/07/02 17:07:59 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-xgboost, version 2
Created version '2' of model 'nyc-taxi-xgboost'.


<ModelVersion: aliases=[], creation_timestamp=1751447279837, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1751447279837, metrics=None, model_id=None, name='nyc-taxi-xgboost', params=None, run_id='a297a9bc617941519486bac78bd4567a', run_link='', source='models:/m-cffe91d66ddc4191b7e993c1dd0e2daf', status='READY', status_message=None, tags={}, user_id='', version='2'>